要開發一個可以連接 mongodb 的 python 程式之前，需要先安裝 mongodb 的連接套件，使用下面的指令安裝 python3 連接 mongodb 的模組套件。
#pip3 install pymongo

安裝完 pymongo 之後，輸入底下的程式，準備抓取 pi 上面的 flask api，並將結果儲存至 mongodb 內

In [ ]:
from pymongo import MongoClient  
import requests
import base64
import datetime
import time
import json

def clear():
  clear_url = url+'/clear'
  r = requests.get(clear_url) # 將pi flask web server上的辨識結果與影像清除
  if r.status_code == requests.codes.ok: # 確定連接是否順利 (not 502 or 400)
      print("clear_OK")

def getImg(j, filename):   # 從flask取得推論結果
  img_url = url+'/img?file='+filename # 指定web中影像的位址
  # 下載圖片並儲存成一個檔案
  with open('static/'+filename, 'wb') as handle: #使用python內建的檔案讀寫工具open
        response = requests.get(img_url, stream=True)
        if not response.ok:
            print (response)

        for block in response.iter_content(1024): #將影像儲存在檔案裡
            if not block:
                break
            handle.write(block)

  with open('static/'+filename, "rb") as f: #使用python內建的檔案讀寫工具open
    bin_pic = base64.b64encode(f.read()).decode('utf-8') # 將讀取到的圖片(f)編碼成二進制檔
    mydata = {'label':j['label'], 'x':j['x'], 'y':j['y'], 'w':j['w'], 'h':j['h'], 'img': filename, 'image_data': bin_pic}
    result = coll.insert_one(mydata) # 將影像結果編碼之後，儲存在 mongodb內

def fetchYoloResult():  # 從flask取得推論結果
  datetime_dt = datetime.datetime.today()# 獲得當地時間
  datetime_str = datetime_dt.strftime("%Y/%m/%d %H:%M:%S")  # 格式化日期
  print ("fetchYoloResult:"+datetime_str)
  r = requests.get(url+'/list')
  if r.status_code == requests.codes.ok:
      print(r.text)

  #將 pi flask web server上面的影像辨識結果list，轉換成json物件
  yolo_result = r.text
  yolo_json = json.loads(yolo_result)

  for j in yolo_json:
      print(j) #把每一個json物件印出來檢查
#      coll.insert_one(j) #把json物件新增到 mongodb
      img=j['img'] # 取出 json物件中影像檔案名稱
      getImg(j, img)  # 至 pi flask web server取出影像內容儲存至mongodb

  clear() # 將pi flask web server上的資料清除

url = 'http://10.2.6.82:5000' # 指定pi flask web的位址
# 連接後端 server 的 mongodb
client = MongoClient(host = '127.0.0.1', port = 27017)
db = client['yolo_pred_db'] # 建立一個data base 命名為yolo_pred_db
coll = db['Collections'] #建立一個collections，透過coll變數呼叫他

while True:
	fetchYoloResult()
	time.sleep(5) # 每隔5秒抓一次資料
  

將檔案儲存為 backMonogoServer.py ，然後使用　python3 backMongoServer.py ，程式會每隔五秒鐘去pi上面抓取資料。